# Etapa 2
- Calculo da Entropia e métricas.
- Criação do agente de reforço para seleção da próxima rotulação.

In [8]:
from tqdm import tqdm
import numpy as np
from tqdm import tqdm
from tqdm.notebook import tqdm
from pathlib import Path
from ultralytics import YOLO
from ultralytics.engine.results import Results
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
import cv2
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback
from typing import List, Dict, Tuple

In [2]:
model = YOLO("runs/detect/yolov11-initial/weights/best.pt") 

In [4]:

metrics = model.val(
    data="coco.yaml",
    split="val",
    imgsz=480,
    conf=0.5,  # Limiar de confiança
    iou=0.6, # Limiar de NMS
)


Ultralytics 8.3.135  Python-3.13.2 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060, 8192MiB)
YOLO11x summary (fused): 190 layers, 56,919,424 parameters, 0 gradients, 194.9 GFLOPs
val: Fast image access  (ping: 0.30.0 ms, read: 136.930.6 MB/s, size: 161.5 KB)


val: Scanning E:\COCO-Dataset\val2017\val\labels.cache... 4000 images, 39 backgrounds, 0 corrupt: 100%|██████████| 4000/4000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 250/250 [02:31<00:00,  1.65it/s]


                   all       4000      29401      0.793      0.519      0.667       0.54
                person       2167       8916      0.935      0.569      0.756      0.629
               bicycle        118        250      0.816      0.516      0.681      0.453
                   car        432       1587      0.876      0.452      0.674      0.528
            motorcycle        132        311      0.837      0.627      0.759      0.577
              airplane         80        117      0.868      0.786      0.868      0.749
                   bus        153        239      0.775       0.72      0.791       0.71
                 train        134        160      0.968      0.756      0.872      0.753
                 truck        198        318      0.503      0.557       0.53      0.429
                  boat         97        341       0.77      0.372      0.575      0.379
         traffic light        146        475      0.782      0.356      0.573      0.387
          fire hydran

In [5]:
metrics

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000022BCFC05940>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0

In [3]:
def calculate_entropy(probabilities):
    """Calcula a entropia de Shannon para um vetor de probabilidades."""
    epsilon = 1e-10  # Evitar log(0)
    return -np.sum(probabilities * np.log(probabilities + epsilon), axis=-1)

In [4]:
class CocoValDataset(Dataset):
    def __init__(self, base_path, img_size=480):
        self.img_dir = Path(base_path) / 'val' / 'images'
        self.img_files = list(self.img_dir.glob('*.jpg'))
        
        # Transformações ajustadas
        self.transform = transforms.Compose([
            transforms.ToTensor(),         # Converte para tensor [0.0, 1.0] e divide por 255
            transforms.Resize((img_size, img_size)),
            
        ])

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_path = self.img_files[idx]
        
        # Carregar com OpenCV e converter para RGB
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Converter para PIL Image para garantir compatibilidade
        from PIL import Image
        img_pil = Image.fromarray(img)
        
        # Aplicar transformações
        tensor_img = self.transform(img_pil)  # Agora em [0.0, 1.0]
        
        return tensor_img, img_path.name

    def calculate_statistics(self):
        """Calcula média e desvio padrão do dataset inteiro"""
        # Inicializar acumuladores
        mean_sum = torch.zeros(3)
        std_sum = torch.zeros(3)
        total_pixels = 0
        
        # Iterar por todas as imagens
        for img_path in tqdm(self.img_files, desc='Calculando estatísticas'):
            img = cv2.imread(str(img_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            tensor_img = self.base_transform(img)  # Tensor [3, H, W]
            
            # Acumular estatísticas
            mean_sum += tensor_img.sum(dim=[1, 2])  # Soma por canal
            std_sum += (tensor_img ** 2).sum(dim=[1, 2])  # Soma quadrados
            total_pixels += tensor_img.shape[1] * tensor_img.shape[2]  # H * W
        
        # Calcular valores finais
        mean = mean_sum / total_pixels
        std = torch.sqrt((std_sum / total_pixels) - (mean ** 2))
        
        return mean.tolist(), std.tolist()



In [5]:
def validate_model(model, device, data_path, batch_size=8):
    dataset = CocoValDataset(data_path)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    model.eval()
    all_confs = []
    all_entropies = []
    
    # Configurações do progresso
    pbar_format = "Validando: {percentage:.0f}%|{bar}| {n_fmt}/{total_fmt} [Tempo: {elapsed}<{remaining}]"
    
    with torch.no_grad():
        with tqdm(total=len(loader), 
                 desc='Validação', 
                 bar_format=pbar_format,
                 dynamic_ncols=True,  # Ajusta automaticamente ao tamanho do notebook
                 mininterval=0.5,     # Atualiza a cada 0.5 segundos
                 leave=True) as pbar:
            
            for batch, img_names in loader:
                batch = batch.to(device)
                
                # Inferência do modelo
                results = model(batch, save_conf=True)
                
                # Processamento dos resultados
                for result in results:
                    if hasattr(result, 'boxes') and result.boxes is not None:
                        confs = result.boxes.conf.cpu().numpy()
                        entropies = [calculate_entropy(c) for c in confs]
                        
                        all_confs.extend(confs.tolist())
                        all_entropies.extend(entropies)
                
                # Atualização dinâmica da barra
                pbar.update(1)
                pbar.refresh()  # Força atualização imediata
    
    # Cálculo final das métricas
    avg_conf = np.mean(all_confs) if all_confs else 0
    avg_entropy = np.mean(all_entropies) if all_entropies else 0
    
    print(f'\n✅ Validação Completa:')
    print(f'  - Confiança Média: {avg_conf:.4f}')
    print(f'  - Entropia Média: {avg_entropy:.4f}')
    
    return avg_conf, avg_entropy


model.to('cuda')
    
data_path = 'E:/COCO-Dataset/val2017/'
avg_conf, avg_entropy = validate_model(model, 'cuda', data_path)

Validando: 0%|          | 0/500 [Tempo: 00:00<?]


0: 480x480 1 person, 1 bottle, 4 chairs, 1 potted plant, 1 dining table, 3 tvs, 1 refrigerator, 5 vases, 27.7ms
1: 480x480 1 bear, 27.7ms
2: 480x480 1 bottle, 1 chair, 2 potted plants, 1 bed, 26 books, 27.7ms
3: 480x480 1 truck, 2 stop signs, 27.7ms
4: 480x480 3 ovens, 2 refrigerators, 27.7ms
5: 480x480 2 persons, 1 sports ball, 27.7ms
6: 480x480 9 persons, 1 tennis racket, 1 chair, 27.7ms
7: 480x480 11 persons, 2 backpacks, 2 handbags, 1 tennis racket, 27.7ms
Speed: 0.1ms preprocess, 27.7ms inference, 31.6ms postprocess per image at shape (1, 3, 480, 480)

0: 480x480 4 persons, 2 cell phones, 26.0ms
1: 480x480 7 persons, 26.0ms
2: 480x480 1 bowl, 1 sandwich, 26.0ms
3: 480x480 2 persons, 1 horse, 2 cows, 1 surfboard, 26.0ms
4: 480x480 6 cars, 4 trucks, 26.0ms
5: 480x480 2 persons, 3 buss, 26.0ms
6: 480x480 2 cats, 3 laptops, 1 keyboard, 26.0ms
7: 480x480 2 airplanes, 26.0ms
Speed: 0.0ms preprocess, 26.0ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 480)

0: 480x480 1 c

In [6]:
print(avg_conf, avg_entropy)

0.6079268866168356 0.25778666


In [65]:
# Configurações
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
class YOLORLEnv(gym.Env):
    """Ambiente RL para Active Learning usando Entropia e Confiança do YOLOv11."""
    
    def __init__(self, yolo_model: YOLO,
        unlabeled_images: List[str],
        oracle_labels: Dict[str, list],  # Rótulos pré-definidos (ex: COCO)
        batch_size: int = 32,
        initial_mAP: float = 0.3,
    ):
        super(YOLORLEnv, self).__init__()
        
        self.yolo = yolo_model
        self.unlabeled_images = unlabeled_images.copy()
        self.oracle = oracle_labels
        self.batch_size = batch_size
        self.current_mAP = initial_mAP
        
        # Espaço de observação: [entropia, confiança_média]
        self.observation_space = gym.spaces.Box(
            low=0.0, 
            high=1.0, 
            shape=(2,), 
            dtype=np.float32
        )
        
        # Ação: 0 = não rotular, 1 = rotular
        self.action_space = gym.spaces.Discrete(2)  
        
        # Estado atual
        self.current_image_idx = 0
        self.current_image_path = None

    def reset(self, **kwargs) -> np.ndarray:
        """Reinicia o ambiente com uma nova imagem."""
        self.current_image_idx = np.random.randint(0, len(self.unlabeled_images))
        self.current_image_path = self.unlabeled_images[self.current_image_idx]
        
        # Extrair entropia e confiança
        entropy, avg_confidence = self._extract_metrics(self.current_image_path)
        state = np.array([entropy, avg_confidence], dtype=np.float32)

        # Normalizar entropia (supondo máximo teórico de log(num_classes))
        max_entropy = np.log(self.yolo.model.nc)
        entropy_norm = entropy / max_entropy
        
        # Confiança já está entre 0 e 1
        state = np.array([entropy_norm, avg_confidence], dtype=np.float32)
        
        # Clipar para evitar outliers
        state = np.clip(state, 0.0, 1.0)
        
        return state, {}

    def step(self, action: int) -> Tuple[np.ndarray, float, bool, dict]:
        """Executa uma ação e retorna próximo estado, recompensa, etc."""
        reward = 0.0
        done = False
        
        if action == 1:  # Rotular a imagem
            # Verificar se a imagem atual é válida
            entropy, avg_confidence = self._extract_metrics(self.current_image_path)
            if entropy == 0.0 and avg_confidence == 0.0:
                print(f"Imagem {self.current_image_path} pulada (sem detecções).")
                reward = -0.5  # Penalizar ação de rotular imagem inválida
            else:
                label = self.oracle.get(self.current_image_path, None)
                if label is not None:
                    self._add_to_training_data(self.current_image_path, label)
                    new_mAP = self._evaluate_model()
                    reward = new_mAP - self.current_mAP
                    self.current_mAP = new_mAP
                    self.unlabeled_images.pop(self.current_image_idx)
        
        # Próximo estado
        next_state, _ = self.reset()
        
        # Terminar episódio após processar um batch
        done = len(self.unlabeled_images) % self.batch_size == 0
        
        return next_state, reward, done, False, {}

    def _extract_metrics(self, image_path: str) -> Tuple[float, float]:
        """Extrai entropia e confiança média da imagem."""
        results = self.yolo.predict(image_path, verbose=False)
        
        if len(results) == 0:
            return 0.0, 0.0  # Caso sem detecções
        
        confs = results[0].boxes.conf.cpu().numpy()
        print(f"AS CONFS AQUI: {confs}")

        # Caso 1: Sem detecções ou resultados inválidos
        if len(results[0].boxes) == 0 or results[0].boxes.conf is None:
            return 0.0, 0.0  # Valores padrão

        try:
            # Calcular entropia
            entropy = -np.sum(confs * np.log(confs + 1e-10), axis=-1)

            print(f"ENTROPIA {entropy}")
            
            # Calcular confiança média (média das confianças das detecções)
            avg_confidence = np.clip(confs.mean(), 0.0, 1.0)
            
            return entropy, avg_confidence
        except Exception as e:
            print(f"Erro ao processar {image_path}: {e}")
            return 0.0, 0.0
        

    def _add_to_training_data(self, image_path: str, label: list):
        """Adiciona imagem e rótulo ao dataset de treino."""
        # Implemente conforme seu formato de dados (ex: YOLO TXT)
        pass

    def _evaluate_model(self) -> float:
        """Avaliação simplificada do modelo (substitua por uma lógica real)."""
        return self.current_mAP + np.random.uniform(0.0, 0.05)
    


In [66]:
  # 1. Carregar YOLOv11
yolo = YOLO("runs/detect/yolov11-initial/weights/best.pt").to(DEVICE)
    
    # 2. Simular dados não rotulados e oráculo (ex: 1000 imagens)
unlabeled_images = [f"E:/COCO-Dataset/val2017/val/images/{f}" for f in os.listdir("E:/COCO-Dataset/val2017/val/images/")][:4000]
oracle_labels = {img: [...] for img in unlabeled_images}  # Preencher com anotações reais
    
# 3. Criar ambiente
env = YOLORLEnv(
        yolo_model=yolo,
        unlabeled_images=unlabeled_images,
        oracle_labels=oracle_labels,
        batch_size=32,
)
env = DummyVecEnv([lambda: env])
    
# 4. Treinar agente PPO
agent = PPO(
        "MlpPolicy",
        env,
        verbose=1,
        policy_kwargs=dict(
                net_arch=[64, 64],
                optimizer_kwargs=dict(weight_decay=1e-4)  # Regularização L2
        ),
        learning_rate=1e-4,  # Reduzir taxa de aprendizado
        clip_range=0.2,  # Clipar gradientes
        ent_coef=0.01,  # Incentivar exploração
        device=DEVICE,
        
)
    
agent.learn(total_timesteps=5000)
    
# 5. Salvar modelo
agent.save("yolo_active_learning_simple_agent")

Using cuda device
AS CONFS AQUI: [     0.9581     0.85089]
ENTROPIA 0.17840856313705444
AS CONFS AQUI: [    0.92214     0.64908     0.47887]
ENTROPIA 0.7078866958618164
AS CONFS AQUI: [     0.8977     0.89122     0.88259     0.84593     0.81159     0.78039     0.68051     0.56065     0.54072     0.52793     0.45365     0.29107]
ENTROPIA 2.7880911827087402
AS CONFS AQUI: [     0.8977     0.89122     0.88259     0.84593     0.81159     0.78039     0.68051     0.56065     0.54072     0.52793     0.45365     0.29107]
ENTROPIA 2.7880911827087402
AS CONFS AQUI: [    0.98872     0.98442     0.90282     0.62369     0.31399      0.2863]
ENTROPIA 1.135223388671875
AS CONFS AQUI: [    0.98872     0.98442     0.90282     0.62369     0.31399      0.2863]
ENTROPIA 1.135223388671875
AS CONFS AQUI: [    0.74037     0.70854     0.60273     0.57341     0.49798     0.45476     0.34333]
ENTROPIA 2.163318157196045
AS CONFS AQUI: [    0.74037     0.70854     0.60273     0.57341     0.49798     0.45476     0